In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
            
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 6):  # There are 5 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
 

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('sb_bus_details.csv', index=False)

# Close the driver
driver.quit()

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/kolkata-to-haldia: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7CA7DEEB2+31554]
	(No symbol) [0x00007FF7CA757EE9]
	(No symbol) [0x00007FF7CA61872A]
	(No symbol) [0x00007FF7CA668434]
	(No symbol) [0x00007FF7CA66853C]
	(No symbol) [0x00007FF7CA6AF6A7]
	(No symbol) [0x00007FF7CA68D06F]
	(No symbol) [0x00007FF7CA6AC977]
	(No symbol) [0x00007FF7CA68CDD3]
	(No symbol) [0x00007FF7CA65A33B]
	(No symbol) [0x00007FF7CA65AED1]
	GetHandleVerifier [0x00007FF7CAAE8B2D+3217341]
	GetHandleVerifier [0x00007FF7CAB35AF3+3532675]
	GetHandleVerifier [0x00007FF7CAB2B0F0+3489152]
	GetHandleVerifier [0x00007FF7CA88E786+750614]
	(No symbol) [0x00007FF7CA76376F]
	(No symbol) [0x00007FF7CA75EB24]
	(No symbol) [0x00007FF7CA75ECB2]
	(No symbol) [0x00007FF7CA74E17F]
	BaseThreadInitThunk [0x00007FFAD5D0257D+29]
	RtlUserThreadStart [0x00007FFAD7B6AF28+40]

Error occurred while scraping bus details for https://www.redbus.in/bus-tick

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/digha-to-barasat-west-bengal: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=126.0.6478.183)
Stacktrace:
	GetHandleVerifier [0x00007FF7CA7DEEB2+31554]
	(No symbol) [0x00007FF7CA757EE9]
	(No symbol) [0x00007FF7CA61872A]
	(No symbol) [0x00007FF7CA5ED995]
	(No symbol) [0x00007FF7CA6944D7]
	(No symbol) [0x00007FF7CA6AC051]
	(No symbol) [0x00007FF7CA68CDD3]
	(No symbol) [0x00007FF7CA65A33B]
	(No symbol) [0x00007FF7CA65AED1]
	GetHandleVerifier [0x00007FF7CAAE8B2D+3217341]
	GetHandleVerifier [0x00007FF7CAB35AF3+3532675]
	GetHandleVerifier [0x00007FF7CAB2B0F0+3489152]
	GetHandleVerifier [0x00007FF7CA88E786+750614]
	(No symbol) [0x00007FF7CA76376F]
	(No symbol) [0x00007FF7CA75EB24]
	(No symbol) [0x00007FF7CA75ECB2]
	(No symbol) [0x00007FF7CA74E17F]
	BaseThreadInitThunk [0x00007FFAD5D0257D+29]
	RtlUserThreadStart [0x00007FFAD7B6AF28+40]

Error occu

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/kolkata-to-futishanko: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7CA7DEEB2+31554]
	(No symbol) [0x00007FF7CA757EE9]
	(No symbol) [0x00007FF7CA61872A]
	(No symbol) [0x00007FF7CA668434]
	(No symbol) [0x00007FF7CA66853C]
	(No symbol) [0x00007FF7CA6AF6A7]
	(No symbol) [0x00007FF7CA68D06F]
	(No symbol) [0x00007FF7CA6AC977]
	(No symbol) [0x00007FF7CA68CDD3]
	(No symbol) [0x00007FF7CA65A33B]
	(No symbol) [0x00007FF7CA65AED1]
	GetHandleVerifier [0x00007FF7CAAE8B2D+3217341]
	GetHandleVerifier [0x00007FF7CAB35AF3+3532675]
	GetHandleVerifier [0x00007FF7CAB2B0F0+3489152]
	GetHandleVerifier [0x00007FF7CA88E786+750614]
	(No symbol) [0x00007FF7CA76376F]
	(No symbol) [0x00007FF7CA75EB24]
	(No symbol) [0x00007FF7CA75ECB2]
	(No symbol) [0x00007FF7CA74E17F]
	BaseThreadInitThunk [0x00007FFAD5D0257D+29]
	RtlUserThreadStart [0x00007FFAD7B6AF28+40]

Error occurred while scraping bus details for https://www.redbus.in/bus-

NameError: name 'driver' is not defined

In [2]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Burdwan to Kolkata,https://www.redbus.in/bus-tickets/burdwan-to-k...,Shyamoli Paribahan Pvt Ltd,Volvo A/C Seater (2+2),17:40,02h 50m,20:30,3.9,INR 429,1 Seat available
1,Burdwan to Kolkata,https://www.redbus.in/bus-tickets/burdwan-to-k...,Greenline (Karunamoyee),Volvo A/C Seater (2+2),18:30,02h 50m,21:20,3.5,INR 429,1 Seat available
2,Burdwan to Kolkata,https://www.redbus.in/bus-tickets/burdwan-to-k...,Greenline,Volvo A/C Seater (2+2),18:10,02h 50m,21:00,1.8,INR 429,1 Seat available
3,Burdwan to Kolkata,https://www.redbus.in/bus-tickets/burdwan-to-k...,SHYAMOLI Pvt. Ltd.(Karunamoyee),Volvo A/C Seater (2+2),18:35,02h 20m,20:55,2.3,INR 429,29 Seats available
4,Kolkata to Burdwan,https://www.redbus.in/bus-tickets/kolkata-to-b...,SBSTC-KOLKATA - KANDI - VIA - FUTISANKO - 16:2...,Non AC Seater (2+3),16:25,02h 00m,18:25,3.9,INR 95,48 Seats available
...,...,...,...,...,...,...,...,...,...,...
121,Kolkata to Bajkul (West Bengal),https://www.redbus.in/bus-tickets/kolkata-to-b...,Bengal Surface Transport,A/C Seater Push Back (2+3),23:20,02h 55m,02:15,2.7,INR 419,61 Seats available
122,Kolkata to Bajkul (West Bengal),https://www.redbus.in/bus-tickets/kolkata-to-b...,Basu Travels,A/C Seater Push Back (2+3),23:30,03h 00m,02:30,2.8,INR 300,20 Seats available
123,Kolkata to Bajkul (West Bengal),https://www.redbus.in/bus-tickets/kolkata-to-b...,Bengal Surface Transport,A/C Seater / Sleeper (2+2),23:55,02h 50m,02:45,2.7,INR 339,29 Seats available
124,Kolkata to Bajkul (West Bengal),https://www.redbus.in/bus-tickets/kolkata-to-b...,,,,,,,,
